In [43]:
import json
import sys
import numpy as np
from datetime import datetime

time_format = '%Y-%m-%dT%H:%M:%SZ'

allRuns = ['config_3/results_parsec_run1.json', 'config_3/results_parsec_run2.json', 'config_3/results_parsec_run3.json']
total_times = np.zeros((3,7))
for run in allRuns:
    file = open(run, 'r')
    json_file = json.load(file)
    start_times = []
    completion_times = []
    names = []

    pos = allRuns.index(run)
    
    for item in json_file['items']:
        name = item['status']['containerStatuses'][0]['name']
        names.append(name)
        print("Job: ", str(name))
        if str(name) != "memcached":
            try:
                start_time = datetime.strptime(
                        item['status']['containerStatuses'][0]['state']['terminated']['startedAt'],
                        time_format)
                completion_time = datetime.strptime(
                        item['status']['containerStatuses'][0]['state']['terminated']['finishedAt'],
                        time_format)
                print("Job time: ", completion_time - start_time)
                start_times.append(start_time)
                completion_times.append(completion_time)
            except KeyError:
                print("Job {0} has not completed....".format(name))
                sys.exit(0)

    if len(start_times) != 6 and len(completion_times) != 6:
        print("You haven't run all the PARSEC jobs. Exiting...")
        sys.exit(0)
        
    total_times[pos,0:6] = [(x - y).total_seconds() for x,y in zip(completion_times, start_times)]
    total_times[pos,-1] = (max(completion_times) - min(start_times)).total_seconds()
    #print(names)
    #print(total_times)
    print("Total time: {0}".format(max(completion_times) - min(start_times)))
    print("-----------")
    file.close()
    print((max(completion_times) - min(start_times)).total_seconds())
    
# Results for part 3a
print(total_times)
print(names)
means = np.mean(total_times,0)
print(means)
std = np.std(total_times,0)
print(std)


Job:  parsecblackscholes
Job time:  0:01:27
Job:  parseccanneal
Job time:  0:03:22
Job:  parsecdedup
Job time:  0:01:21
Job:  parsecferret
Job time:  0:05:08
Job:  parsecfreqmine
Job time:  0:01:54
Job:  parsec-splash2x-fft
Job time:  0:02:03
Job:  memcached
Total time: 0:07:08
-----------
428.0
Job:  parsecblackscholes
Job time:  0:01:27
Job:  parseccanneal
Job time:  0:03:00
Job:  parsecdedup
Job time:  0:01:35
Job:  parsecferret
Job time:  0:05:20
Job:  parsecfreqmine
Job time:  0:02:08
Job:  parsec-splash2x-fft
Job time:  0:01:40
Job:  memcached
Total time: 0:07:10
-----------
430.0
Job:  parsecblackscholes
Job time:  0:01:27
Job:  parseccanneal
Job time:  0:03:09
Job:  parsecdedup
Job time:  0:01:38
Job:  parsecferret
Job time:  0:05:18
Job:  parsecfreqmine
Job time:  0:01:53
Job:  parsec-splash2x-fft
Job time:  0:01:50
Job:  memcached
Total time: 0:06:52
-----------
412.0
[[ 87. 202.  81. 308. 114. 123. 428.]
 [ 87. 180.  95. 320. 128. 100. 430.]
 [ 87. 189.  98. 318. 113. 110. 4

In [44]:
423/60

7.05